In [ ]:
!pip install transformers -U
!pip install accelerate -U
!pip install kaggle


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.2
    Uninstalling transformers-4.40.2:
      Successfully uninstalled transformers-4.40.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.

In [ ]:
from google.colab import files
uploaded = files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d rmisra/news-category-dataset
!unzip news-category-dataset.zip


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/rmisra/news-category-dataset
License(s): Attribution 4.0 International (CC BY 4.0)
 34% 9.00M/26.5M [00:00<00:00, 59.3MB/s]
100% 26.5M/26.5M [00:00<00:00, 131MB/s] 
Archive:  news-category-dataset.zip
  inflating: News_Category_Dataset_v3.json  


In [ ]:
import pandas as pd
from transformers import GPT2Tokenizer

# Load dataset
data = pd.read_json('/content/News_Category_Dataset_v3.json', lines=True)

# Sample 50% of the dataset to speed up training
data_sample = data.sample(frac=0.5)
data_sample.to_json('sampled_data.json', orient='records', lines=True)

# Initialize tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
from transformers import GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import logging

# Enable logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
logger.info("Model loaded successfully.")

# Prepare dataset
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='sampled_data.json',
    block_size=128
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=0.1,  # Set to 2 epochs
    per_device_train_batch_size=4,
    logging_steps=50,
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# Start training
trainer.train()
logger.info("Training completed.")

# Save the fine-tuned model
model.save_pretrained('./fine_tuned_model')


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
50,2.962900
100,2.639300
150,2.536900
200,2.500700
250,2.468700
300,2.477900
350,2.422100
400,2.429700
450,2.370800
500,2.358000


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
import logging

# Enable logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
logger.info("Model loaded successfully.")

# Assuming your dataset preparation and training code here...

# Save the fine-tuned model
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')
logger.info("Model and tokenizer saved successfully.")


In [ ]:
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer

# Load fine-tuned model
model = GPT2LMHeadModel.from_pretrained('./fine_tuned_model')
tokenizer = GPT2Tokenizer.from_pretrained('./fine_tuned_model')

# Setup generation pipeline
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Test the model with a prompt
prompt = "The latest advancements in artificial intelligence"
generated_text = generator(prompt, max_length=50, truncation=True, clean_up_tokenization_spaces=True)

# Print the generated text
formatted_text = generated_text[0]['generated_text'].replace('\n', ' ').replace('True', '').replace('False', '')
print(formatted_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The latest advancements in artificial intelligence are coming soon, but the question remains...  When will this system become popular? The current system has shown to be capable of delivering a very high throughput of traffic. This number isn't only based on real-


In [ ]:
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer

# Load fine-tuned model
model = GPT2LMHeadModel.from_pretrained('./fine_tuned_model')
tokenizer = GPT2Tokenizer.from_pretrained('./fine_tuned_model')

# Setup generation pipeline
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Test the model with a prompt
prompt = "What does the future hold?"
generated_text = generator(prompt, max_length=100, truncation=True, clean_up_tokenization_spaces=True)

# Process the generated text to format each sentence on a new line
raw_text = generated_text[0]['generated_text']
sentences = raw_text.split('. ')
formatted_text = '.\n'.join(sentences)

# Print the formatted text
print(formatted_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What does the future hold?

We are not yet as optimistic as we were when the first phase began, but we expect them to come together to form one entity with both a clear roadmap and strong foundation in place.
We know that the company is still in the early phases of product development.
We want to be in better shape to develop our products and will work with the potential partners to ensure the success of the product and service.

But the reality is that we cannot solve long-


In [ ]:
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer

# Load fine-tuned model
model = GPT2LMHeadModel.from_pretrained('./fine_tuned_model')
tokenizer = GPT2Tokenizer.from_pretrained('./fine_tuned_model')

# Setup generation pipeline
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Test the model with a prompt
prompt = "The mustang goes.."
generated_text = generator(prompt, max_length=100, truncation=True, clean_up_tokenization_spaces=True)

# Process the generated text to format each sentence on a new line
raw_text = generated_text[0]['generated_text']
sentences = raw_text.split('. ')
formatted_text = '.\n'.join(sentences)

# Print the formatted text
print(formatted_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The mustang goes.." ―Aang to Toph [src]

Aang was one of the seven main characters of the final version of the series based at Marvel Studios.
Following the destruction of the Marvel Cinematic Universe in "The Avengers" (which the titular story is set in), Anahida became The Watcher's wife and "King" (who serves as the central figure throughout the movie) would continue to rule a feudal feudal family in the United States known as


In [ ]:
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer

# Load fine-tuned model
model = GPT2LMHeadModel.from_pretrained('./fine_tuned_model')
tokenizer = GPT2Tokenizer.from_pretrained('./fine_tuned_model')

# Setup generation pipeline
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Test the model with a prompt
prompt = "The year is 2100"
generated_text = generator(prompt, max_length=100, truncation=True, clean_up_tokenization_spaces=True)

# Process the generated text to format each sentence on a new line
raw_text = generated_text[0]['generated_text']
sentences = raw_text.split('. ')
formatted_text = '.\n'.join(sentences)

# Print the formatted text
print(formatted_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The year is 2100, when a solar eclipse may occur, where solar activity should be lessened by about 1%-2%.
However, this may not all be possible.
Some countries have already decided to ban solar power generation after several decades because of concerns about the impacts of high heat in low temperature regions where solar power stations have long been built and where solar output is much lower than those provided by the large solar power manufacturers.
There are concerns, as well as a great deal of evidence that some countries


In [ ]:
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer

# Load fine-tuned model
model = GPT2LMHeadModel.from_pretrained('./fine_tuned_model')
tokenizer = GPT2Tokenizer.from_pretrained('./fine_tuned_model')

# Setup generation pipeline
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Test the model with a prompt
prompt = "Predict the future"
generated_text = generator(prompt, max_length=500, truncation=True, clean_up_tokenization_spaces=True)

# Process the generated text to format each sentence on a new line
raw_text = generated_text[0]['generated_text']
sentences = raw_text.split('. ')
formatted_text = '.\n'.join(sentences)

# Print the formatted text
print(formatted_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Predict the future, it's likely to be a slightly more complex situation.
For starters, not all systems will have the capacity to forecast the future.

In some cases, you will be able to extrapolate the probability of a given outcome.
Most cases will be very small, so it's possible for some changes to prove less likely than others (for example, a certain level of confidence in the outcome of a high-risk experiment with an unknown number of participants might be too high).
In some cases, however, you might just have an easier time forecasting when that happens.

If you have a large project to run on your personal computer, you can use a software tool that you download and use with the help of software updates.
If you do not have a reliable source of updates, for example, you'll need to look elsewhere.

To get started with a system like this, it is important to give a good reason why your plan might not go forward.
You should also keep in mind that the process is different for every syste

In [ ]:

from google.colab import drive
drive.mount('/content/drive')


